In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from transformers import AutoModel, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch
import os

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # ✅ Use 4-bit instead of 8-bit
    bnb_4bit_quant_type="nf4",  # Normalized Float 4
    bnb_4bit_use_double_quant=True,  # Use double quantization
    bnb_4bit_compute_dtype=torch.bfloat16,  # Compute in bfloat16 for efficiency
)

model = LlamaForCausalLM.from_pretrained("borko_1", quantization_config = quantization_config).to("cuda")
tokenizer = AutoTokenizer.from_pretrained('borko_1_tok')


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.save_pretrained("borko_1_torch")

: 

In [10]:
from huggingface_hub import create_repo

repo_name = "borko-1"
username = "SookX"  

create_repo(repo_id=f"{username}/{repo_name}", exist_ok=True)

RepoUrl('https://huggingface.co/SookX/borko-1', endpoint='https://huggingface.co', repo_type='model', repo_id='SookX/borko-1')

In [11]:
from huggingface_hub import HfApi, HfFolder, Repository
from transformers import AutoTokenizer, AutoModelForCausalLM

repo_name = "SookX/borko-1"
model.save_pretrained(repo_name)
tokenizer.save_pretrained(repo_name)

# Push to Hugging Face Hub
from huggingface_hub import upload_folder
upload_folder(folder_path=repo_name, repo_id=repo_name, repo_type="model")

adapter_model.safetensors:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SookX/borko-1/commit/7d98343eace4ad2438350f8b1d9535ce4120fca5', commit_message='Upload folder using huggingface_hub', commit_description='', oid='7d98343eace4ad2438350f8b1d9535ce4120fca5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SookX/borko-1', endpoint='https://huggingface.co', repo_type='model', repo_id='SookX/borko-1'), pr_revision=None, pr_num=None)

OSError: deepseek-ai/deepseek-llm-7b-chat is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [15]:
messages = [
    {
        "role": "system",
        "content": "Ти си Български Гласов Асистент, говори само на български език, без да повтаряш каквото и да било вече казано, и твоето име е Борко. Твоят създател е великият Васил Василев",
    },
    {"role": "user", "content": "Кой е твоят създател"},
]
input_tensor = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
)

attention_mask = input_tensor.ne(tokenizer.pad_token_id)  # Mask non-padding tokens

outputs = model.generate(
    input_tensor.to(model.device),
    attention_mask=attention_mask.to(model.device),  # Pass attention mask
    max_new_tokens=100,
    pad_token_id = 100001
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
generated_text

TypeError: The current model class (LlamaModel) is not compatible with `.generate()`, as it doesn't have a language model head. Classes that support generation often end in one of these names: ['ForCausalLM', 'ForConditionalGeneration', 'ForSpeechSeq2Seq', 'ForVision2Seq'].